In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
PATH = "../data/Q6"

In [3]:
f = open(PATH + "_data_full.txt", "r")
s = f.read()
s = s.split('\n')
s[:5]

['RELATIVE SYSCALL(args) \t\t = return',
 'dtrace: 813 dynamic variable drops with non-empty dirty list',
 '   92825 gettimeofday(0x700003993D58, 0x0, 0x0)\t\t = 0 0',
 '  458217 psynch_cvsignal(0x7FE8A00E5EC0, 0xBFD00000BFE00, 0xBFD00)\t\t = 257 0',
 '  458225 gettimeofday(0x7000078FA698, 0x0, 0x0)\t\t = 0 0']

In [4]:
s = [line.strip() for line in s]
s = s[2:]
s[:3]

['92825 gettimeofday(0x700003993D58, 0x0, 0x0)\t\t = 0 0',
 '458217 psynch_cvsignal(0x7FE8A00E5EC0, 0xBFD00000BFE00, 0xBFD00)\t\t = 257 0',
 '458225 gettimeofday(0x7000078FA698, 0x0, 0x0)\t\t = 0 0']

In [5]:
s = [re.split('\(|\)|,\s+|\s+', line)[:5] for line in s]

In [6]:
df = pd.DataFrame(columns=['relative_timestamp', 'activity', 'path', 'p2', 'p3'], data=s)
df.drop(['p2', 'p3'], axis=1, inplace=True)
df.index.name = 'pd_timestamp'
df = df.astype("string")

In [7]:
# Filter rows with correct path
df = df[df["path"].str.contains("/IdeaProjects/Comnata")]#[df["path"].str.contains(".kt")]

# Extract case_id and put to the separate column
df["case_id"] = df["path"].str.extract(r"(nmbr\d+)")[0].str.extract(r"(\d+)")

# Create "activity_file" column
df["activity_file"] = df["activity"] + " | " + df["path"].str.extract(r"([A-Za-z]+\.[a-z]+)")[0]

# Remove rows with NaN values in "activity_file" column
df = df[df['activity_file'].notna()]

# Ordering
df = df[['relative_timestamp', 'activity', 'activity_file', 'case_id', 'path']]

In [8]:
df.head()

,relative_timestamp,activity,activity_file,case_id,path
pd_timestamp,,,,,
2097,1675435,access,access | icon.svg,2,"""/Users/vakosta/Developer/IdeaProjects/Comnata..."
2100,1675481,access,access | icon.png,2,"""/Users/vakosta/Developer/IdeaProjects/Comnata..."
2103,1675702,access,access | icon.svg,1,"""/Users/vakosta/Developer/IdeaProjects/Comnata..."
2105,1675739,access,access | icon.png,1,"""/Users/vakosta/Developer/IdeaProjects/Comnata..."
2235,1676976,access,access | icon.svg,3,"""/Users/vakosta/Developer/IdeaProjects/Comnata..."


In [9]:
df.to_csv(PATH + "_data_filtered.csv", sep=',')